# Import 

In [73]:
%matplotlib qt
%gui qt

import re
import os
import sys

import numpy as np
from loguru import logger

import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches


import pyqtgraph as pg
import pyqtgraph.opengl as gl

from collections import defaultdict
from datetime import datetime

import torch
import gpytorch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.interpolate as interp

In [74]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [75]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,1] * np.cos(np.deg2rad(_data[:,2])) 
        _Zimag  = _data[:,1] * np.sin(np.deg2rad(_data[:,2])) 
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

In [76]:


def EIS_recal_ver02(data, _phz_0 = None):
    f_poi = data[0,:]
    # Z_poi = data[1,:] * np.exp(1j*np.deg2rad(data[2,:]))
    Z_poi = data[1,:] + 1j*data[2,:]
    Y_poi = 1/Z_poi

    Rg0 = 1.611e13
    Cp0 = 1.4e-9
    
    _Rg0_rescale = 1/Rg0*np.power(f_poi,1.583)
    _Cp0_rescale = Cp0*np.power(f_poi,0.911)
    Y_org = Y_poi - _Rg0_rescale + 1j*_Cp0_rescale
    # Y_org = Y_poi - _Rg0_rescale 
    # Y_org = Y_poi + 1j*_Cp0_rescale
    # Y_org = Y_poi
    Z_org = 1/Y_org

    # Phz Calibration
    if _phz_0 is None:
        _phz_0 = np.loadtxt("./phz_Calib.txt")
    
    Z_ampC = np.abs(Z_org)
    # Z_phzC = np.angle(Z_org) - _phz_0
    Z_phzC = np.angle(Z_org) - _phz_0

    Z_rec = Z_ampC * np.exp(1j*Z_phzC)

    # C = 5e-10
    Rs0 = 100
    Z_rec = Z_rec - Rs0



    Cp0 = 5e-10
    _Cp0_rescale = Cp0 * f_poi
    Z_rec = 1/(1/Z_rec - 1j * _Cp0_rescale)

    

    # Ls0 = 1.7e-4
    Ls0 = 5e-4
    _Ls0_rescale = Ls0 * f_poi
    Z_rec = Z_rec - 1j * _Ls0_rescale

    # C = 5e-10
    Rs0 = 566
    Z_rec = Z_rec - Rs0
    
    return np.stack([f_poi, np.real(Z_rec), np.imag(Z_rec)], axis=1).T
    

## Data Loader

In [77]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

# rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
# ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/06017758_归档"
# ch_id = 96     # Perfect of Perfect

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run Without Error

# rootPath = "D:\Baihm\EISNN\Archive/22037380_归档"
# ch_id = 20     # Connection Error


# freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)
freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
# freq_list_DTW = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
EISDict = gatherCSV(rootPath)
chData_full = readChannel(ch_id, EISDict)

# chData_DTW = chData_full[:,:,freq_list_DTW]

if True:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData_full)[0]):
        ch_eis = EIS_recal_ver02(chData_full[i,:,:], phz_calibration)
        chData_full[i,:,:] = ch_eis

# chData = chData[:,:,91:100]
chData = chData_full[:,:,freq_list]


np.shape(chData)
        

2025-05-08 16:51:29.763 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241022_01
2025-05-08 16:51:29.764 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-08 16:51:29.765 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-08 16:51:29.766 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-08 16:51:29.766 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-08 16:51:29.767 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241024_01
2025-05-08 16:51:29.768 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-08 16:51:29.769 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-08 16:51:29.769 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-08 16:51:29.770 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-08 16:51:29.771 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241028_01
2025-05-08 16:51:29.771 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-08 16:51:29.772 | INFO    

(13, 3, 101)

## Data Cleaning

In [78]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from  Outlier import OutlierDetection

CLEAN_FLAG = True
if CLEAN_FLAG:
    import joblib
    weirdSVMmodel = joblib.load("../Outlier/weirdSVMmodel.pkl")
    # eis_seq, eis_cluster, eis_anomaly, leaf_anomaly= OutlierDetection.OutlierDetection(chData_full)
    eis_seq, eis_cluster, eis_anomaly, leaf_anomaly, seq_weird = OutlierDetection.OutlierDetection_Ver02(chData_full, weirdSVMmodel)
else: 
    eis_seq = np.arange(np.shape(chData)[0])

## Data Plot

In [79]:
if True:
    fig= plt.figure(figsize=(15,8), constrained_layout=False)
    axis = [0] * 8
    axis[0] = fig.add_subplot(2,4,1, projection='3d')   
    axis[1] = fig.add_subplot(2,4,2)            
    axis[2] = fig.add_subplot(2,4,3)         
    axis[3] = fig.add_subplot(2,4,4)      
    axis[4] = fig.add_subplot(2,4,5, projection='3d')      
    axis[5] = fig.add_subplot(2,4,6)         
    axis[6] = fig.add_subplot(2,4,7)         
    axis[7] = fig.add_subplot(2,4,8)    

    init_elev = 21  # 仰角
    init_azim = 55  # 方位角
    axis[0].view_init(elev=init_elev, azim=init_azim)
    axis[4].view_init(elev=init_elev, azim=init_azim)


    num_samples = np.shape(chData)[0]

    _x = np.arange(num_samples)[eis_seq]
    _y = np.log10(chData[0,0,:]).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis[0].plot_surface(X, Y, np.log10(np.abs(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)
    axis[4].plot_surface(X, Y, np.rad2deg(np.angle(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)



    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(eis_seq)):
        _x = eis_seq[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(_x/num_samples)
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{i:02d}")
        axis[5].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{i:02d}")


    cmap = plt.colormaps.get_cmap('Set1')
    for i in range(len(eis_seq)):
        _x = eis_seq[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(eis_cluster[i])
        axis[2].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")
        axis[6].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")

    _legend_handle = []
    for i in range(len(np.unique(eis_cluster))):
        _legend_handle.append(mpatches.Patch(color = cmap(i), label = f"{chr(ord('A')+i)}:{len(eis_cluster[eis_cluster==i])}"))
    axis[2].legend(handles=_legend_handle)

    axis[2].sharex(axis[1])
    axis[6].sharex(axis[5])


    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(eis_anomaly)):
        _x = eis_anomaly[i]
        ch_eis = chData[_x,:,:]
        _color = cmap(_x/num_samples)
        axis[3].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{_x:02d}")
        axis[7].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{_x:02d}")
    axis[3].legend()
    axis[3].sharex(axis[1])
    axis[7].sharex(axis[5])



# GPR

## EIS-GP Model

In [103]:
# 之前发现 likelihood声明的时候没加rank，可能会显著影响结果，这里
# from gpytorch.kernels import MultitaskKernel, RQKernel, RBFKernel, MaternKernel

class EISGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_tasks):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_tasks
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            # gpytorch.kernels.RBFKernel(),
            # gpytorch.kernels.RQKernel(),
            # gpytorch.kernels.LinearKernel(),
            # gpytorch.kernels.PolynomialKernel(power=3.0),
            # gpytorch.kernels.PiecewisePolynomialKernel(),
            # gpytorch.kernels.SpectralMixtureKernel(num_mixtures=3),
            # gpytorch.kernels.CosineKernel(),
            
            gpytorch.kernels.MaternKernel(nu=0.5), 
            
            num_tasks=num_tasks, 
            rank=2
        )
        # self.covar_module.data_covar_module.lengthscale = 1
        # self.covar_module.data_covar_module.alpha = 0.001

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


def EISGPTrain(x_train, y_train, x_eval, cluster_id, device, training_iter = 200, lr = 0.05):
    num_tasks = y_train.shape[1]
    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(
        num_tasks=num_tasks, rank = 0).to(device)
    model = EISGPModel(x_train, y_train, likelihood, num_tasks).to(device)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # logger.info(f"Training for {training_iter} iterations...")
    loss_inst       = []
    length_inst     = []
    noise_inst      = []
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(x_train)
        loss = -mll(output, y_train)
        loss.backward()
        optimizer.step()
        
        poi_noise   = model.likelihood.noise.detach().cpu().numpy()
        poi_length  = model.covar_module.data_covar_module.lengthscale.detach().cpu().numpy()
        # poi_length  = 0
        
        loss_inst.append(loss.item())
        noise_inst.append(poi_noise)
        length_inst.append(poi_length)
        if not (i+1)%100:
            logger.info(f"C{cluster_id} - Iter {i+1}/{training_iter}\tLoss: {loss.item()}")
            
    # logger.info("Model Training Finished.")

    # Make predictions
    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.cholesky_jitter(1e-4):
        pred = likelihood(model(x_eval))
    # logger.info("Model Evaluation Finished.")

    return [pred, np.array(loss_inst), np.array(length_inst), np.array(noise_inst)]


## Input Layer

In [81]:
def piecewise_interp(chData, eis_seq, run_list, eis_cluster = None, SPEED_RATE = 1, LOG_FLAG = True):
    # Init xy according to the datetime
    x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    x_day = [x_day[i] for i in eis_seq]

    x_train_full = np.array([(poi - x_day[0]).days for poi in x_day])
    x_eval_full = np.linspace(0,max(x_train_full),max(x_train_full)*SPEED_RATE+1)

    y_train_full = np.stack([chData[eis_seq,1,:],chData[eis_seq,2,:]], axis=2)
    y_train_full = y_train_full.take(run_list, axis=1)


    if LOG_FLAG:
        y_train_log = np.log(y_train_full[:,:,0] + 1j*y_train_full[:,:,1])
        y_train_full = np.stack([y_train_log.real, y_train_log.imag], axis=2)


    # Segmentation of clusters
    if eis_cluster is None:
        eis_cluster = np.zeros_like(eis_seq)
    unique_clusters = np.unique(eis_cluster)
    n_clusters = len(unique_clusters)

    train_mask_list = []
    eval_mask_list = []

    for i in range(n_clusters):
        # 取当前状态和下一个状态的数据
        train_mask = (eis_cluster == unique_clusters[i])
        if i == n_clusters - 1:
            x_eval_end = x_eval_full.max() + 1
        else:
            x_eval_end = x_train_full[(eis_cluster == unique_clusters[i+1])].min()
        # x_state = x_train[state_mask]
        # y_state = y_train[:,state_mask]

        eval_mask = (x_eval_full >= x_train_full[train_mask].min()) & (x_eval_full < x_eval_end)
        
        train_mask_list.append(train_mask)
        eval_mask_list.append(eval_mask)
    
    return x_train_full, y_train_full, x_eval_full, n_clusters, train_mask_list, eval_mask_list
    

def GPDataLoader(x_train, y_train, x_eval, NORM_X_FLAG = True, NORM_Y_FLAG = True):

    Scaler_X        = StandardScaler()
    Scaler_Y_real   = StandardScaler()
    Scaler_Y_imag   = StandardScaler()

    if NORM_Y_FLAG:
        y_train[:,:,0] = Scaler_Y_real.fit_transform(y_train[:,:,0])
        y_train[:,:,1] = Scaler_Y_imag.fit_transform(y_train[:,:,1])
    if NORM_X_FLAG:
        x_train = Scaler_X.fit_transform(x_train.reshape(-1, 1)).flatten()
        x_eval = Scaler_X.transform(x_eval.reshape(-1, 1)).flatten()

    
    y_train = np.hstack((y_train[:,:,0], y_train[:,:,1]))

    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)}")

    return x_train, y_train, x_eval, [Scaler_X, Scaler_Y_real, Scaler_Y_imag]


def GPDataExporter(x_train, y_train, x_eval, y_eval_mean, y_eval_var, ScalerSet, NORM_X_FLAG, NORM_Y_FLAG):
    # Export Data
    n_freq = np.shape(y_train)[1]//2 
    y_train = np.stack((y_train[:,:n_freq], y_train[:,n_freq:]), axis=2)
    y_eval_mean = np.stack((y_eval_mean[:,:n_freq], y_eval_mean[:,n_freq:]), axis=2)
    y_eval_var = np.stack((y_eval_var[:,:n_freq], y_eval_var[:,n_freq:]), axis=2)
    if NORM_X_FLAG:
        x_train = ScalerSet[0].inverse_transform(x_train.reshape(-1, 1)).flatten()
        x_eval = ScalerSet[0].inverse_transform(x_eval.reshape(-1, 1)).flatten()
    
    if NORM_Y_FLAG:
        y_train_real = ScalerSet[1].inverse_transform(y_train[:,:,0])
        y_train_imag = ScalerSet[2].inverse_transform(y_train[:,:,1])
        
        y_eval_mean_real = ScalerSet[1].inverse_transform(y_eval_mean[:,:,0])
        y_eval_mean_imag = ScalerSet[2].inverse_transform(y_eval_mean[:,:,1])

        y_eval_var_real = y_eval_var[:,:,0] * ScalerSet[1].var_
        y_eval_var_imag = y_eval_var[:,:,1] * ScalerSet[2].var_
    else:
        y_train_real = y_train[:,:,0]
        y_train_imag = y_train[:,:,1]

        y_eval_mean_real = y_eval_mean[:,:,0]
        y_eval_mean_imag = y_eval_mean[:,:,1]

        y_eval_var_real = y_eval_var[:,:,0]
        y_eval_var_imag = y_eval_var[:,:,1]

    y_train = np.stack([y_train_real, y_train_imag], axis=2)
    y_eval = np.stack([y_eval_mean_real, y_eval_mean_imag], axis=2)
    y_eval_err = np.stack([y_eval_var_real, y_eval_var_imag], axis=2)
    
    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)} \ny_pred{np.shape(y_eval)} \ny_pred{np.shape(y_eval_err)}")

    return x_train, y_train, x_eval, y_eval, y_eval_err





## Full Run

### Run

In [96]:

SPEED_RATE=2
LOG_FLAG=False
NORM_X_FLAG=True
NORM_Y_FLAG=True


# run_list = range(60,80)
run_list = range(np.shape(chData)[2])

x_train_full, y_train_full, x_eval_full,  n_clusters, train_mask_list, eval_mask_list = \
    piecewise_interp(chData, eis_seq, run_list, 
                     eis_cluster = None, SPEED_RATE = SPEED_RATE, LOG_FLAG=LOG_FLAG)


In [97]:

x_train = x_train_full[train_mask_list[0]]
y_train = y_train_full[train_mask_list[0],:,:]
x_eval = x_eval_full[eval_mask_list[0]]

x_train, y_train, x_eval, ScalerSet = \
    GPDataLoader(x_train, y_train, x_eval, 
        NORM_X_FLAG=NORM_X_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train_tensor = torch.from_numpy(x_train).float().to(device)
x_eval_tensor = torch.from_numpy(x_eval).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)



2025-05-08 16:54:31.008 | INFO     | __main__:GPDataLoader:61 - 
x: (10,) 
y: (10, 202) 
x_pred(37,)


In [98]:


y_eval_tensor, loss_inst, length_inst, noise_inst = \
    EISGPTrain(x_train_tensor, y_train_tensor, x_eval_tensor, 0, device, training_iter=500)


y_eval_mean = y_eval_tensor.mean.cpu().numpy()
y_eval_cov = y_eval_tensor.covariance_matrix.cpu().detach().numpy()
y_eval_var = y_eval_tensor.variance.detach().cpu().numpy()


2025-05-08 16:54:33.502 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 100/500	Loss: -0.2840431332588196
2025-05-08 16:54:35.753 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 200/500	Loss: -0.7687401175498962
2025-05-08 16:54:38.008 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 300/500	Loss: -0.8300940990447998
2025-05-08 16:54:40.243 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 400/500	Loss: -0.8434693813323975
2025-05-08 16:54:42.483 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 500/500	Loss: -0.8493403792381287


In [99]:
x_train, y_train, x_eval, y_eval, y_eval_err = \
        GPDataExporter(x_train, y_train, x_eval, y_eval_mean, y_eval_var, ScalerSet,
                       NORM_X_FLAG=NORM_X_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)
    

2025-05-08 16:54:42.589 | INFO     | __main__:GPDataExporter:99 - 
x: (10,) 
y: (10, 101, 2) 
x_pred(37,) 
y_pred(37, 101, 2) 
y_pred(37, 101, 2)


### Loss Plot

In [100]:
if True:
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.plot(loss_inst)
    ax1.set_title("Loss")


    cmap = plt.colormaps.get_cmap('viridis_r')
    for i in range(np.shape(length_inst)[1]):
        ax2.plot(noise_inst[:,i].flatten(), color = cmap(i/np.shape(length_inst)[1]), linewidth=2, label=f"LengthScale {i+1}")
        ax3.plot(length_inst[:,i].flatten(), color = cmap(i/np.shape(length_inst)[1]), linewidth=2, label=f"LengthScale {i+1}")
    ax2.set_yscale('log')
    ax3.set_title("LengthScale")

    fig.set_tight_layout(True)

### GPR Result Plot

In [101]:
if True:
    n_freq = np.shape(y_eval)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('viridis_r')
    axis0 = fig.add_subplot(2,2,1)
    axis1 = fig.add_subplot(2,2,2)
    axis2 = fig.add_subplot(2,2,3)
    axis3 = fig.add_subplot(2,2,4)


    # for i in range(n_freq):
    # for i in range(70,80):
    for i in range(np.shape(y_eval)[1]):
        axis0.fill_between(x_eval, y_eval[:,i,0] - 2*np.sqrt(y_eval_err[:,i,0]), y_eval[:,i,0] + 2*np.sqrt(y_eval_err[:,i,0]), 
                        alpha=0.2, color = cmap(run_list[i]/n_freq))
        
        axis1.plot(x_eval, y_eval[:,i,0], color = cmap(i/n_freq))
        axis1.plot(x_train, y_train[:,i,0], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')

        axis2.fill_between(x_eval, y_eval[:,i,1] - 2*np.sqrt(y_eval_err[:,i,1]), y_eval[:,i,1] + 2*np.sqrt(y_eval_err[:,i,1]), 
                        alpha=0.2, color = cmap(run_list[i]/n_freq))
        
        axis3.plot(x_eval, y_eval[:,i,1], color = cmap(i/n_freq))
        axis3.plot(x_train, y_train[:,i,1], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')

    # axis1.sharex(axis0)
    # axis1.sharey(axis0)
    # axis3.sharex(axis2)
    # axis3.sharey(axis2)
    axis0.set_xlabel('x')
    axis0.set_ylabel('y')


### GPR EIS Plot

In [102]:
if True:
    
    n_freq = np.shape(y_eval)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    axis0 = fig.add_subplot(2,3,1)
    axis1 = fig.add_subplot(2,3,2)
    axis2 = fig.add_subplot(2,3,3, projection = '3d')
    axis3 = fig.add_subplot(2,3,4)
    axis4 = fig.add_subplot(2,3,5)
    axis5 = fig.add_subplot(2,3,6, projection = '3d')


    init_elev = 40  # 仰角
    init_azim = 30  # 方位角
    axis2.view_init(elev=init_elev, azim=init_azim)
    axis5.view_init(elev=init_elev, azim=init_azim)


    new_f = chData[0,0,:].take(run_list, axis=0)

    if LOG_FLAG:
        
        y_EIS_train =   np.exp(y_train[:,:,0] + 1j * y_train[:,:,1])
        y_EIS_eval = np.exp(y_eval[:,:,0] + 1j * y_eval[:,:,1])

        # f_plot = range(40,80)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=1)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=1)
        new_f = new_f.take(f_plot, axis=0)

    else:
        y_EIS_train = y_train[:,:,0] + 1j*y_train[:,:,1]
        y_EIS_eval = y_eval[:,:,0] + 1j*y_eval[:,:,1]

        # f_plot = range(40,60)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=1)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=1)
        new_f = new_f.take(f_plot, axis=0)



    for i in range(np.shape(x_eval)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
        
        
    for i in range(np.shape(x_train)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_train[i,:])), 'black', alpha = 0.3)
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_train[i,:])), 'black', alpha = 0.3)
    


    _x = np.arange(np.shape(x_eval)[0])
    _y = np.log10(new_f).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis2.plot_surface(X, Y, np.log10(np.abs(y_EIS_eval[:,:])), cmap='viridis_r', alpha=0.8)
    axis5.plot_surface(X, Y, np.rad2deg(np.angle(y_EIS_eval[:,:])) * 180 / np.pi, cmap='viridis_r', alpha=0.8)




## Segment 

### Run

In [104]:
SPEED_RATE=2
LOG_FLAG=True
NORM_X_FLAG=True
NORM_Y_FLAG=True


# run_list = range(60,80)
run_list = range(np.shape(chData)[2])

x_train_full, y_train_full, x_eval_full,  n_clusters, train_mask_list, eval_mask_list = \
    piecewise_interp(chData, eis_seq, run_list, 
                     eis_cluster, SPEED_RATE = SPEED_RATE, LOG_FLAG=LOG_FLAG)



In [105]:
training_iter = 200


y_eval_full = np.zeros((np.shape(x_eval_full)[0], np.shape(y_train_full)[1], 2))
y_eval_err_full = np.zeros((np.shape(x_eval_full)[0], np.shape(y_train_full)[1], 2))

loss_list = []
noise_list = []
length_list = []


for i in range(n_clusters):
# for i in [1]:

    x_train = x_train_full[train_mask_list[i]]
    y_train = y_train_full[train_mask_list[i],:,:]
    x_eval = x_eval_full[eval_mask_list[i]]

    x_train, y_train, x_eval, ScalerSet = \
        GPDataLoader(x_train, y_train, x_eval, 
            NORM_X_FLAG=NORM_X_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x_train_tensor = torch.from_numpy(x_train).float().to(device)
    x_eval_tensor = torch.from_numpy(x_eval).float().to(device)
    y_train_tensor = torch.from_numpy(y_train).float().to(device)



    y_eval_tensor, loss_inst, length_inst, noise_inst = \
    EISGPTrain(x_train_tensor, y_train_tensor, x_eval_tensor, i, device, training_iter=training_iter)

    loss_list.append(loss_inst)
    noise_list.append(noise_inst)
    length_list.append(length_inst)

    y_eval_mean = y_eval_tensor.mean.cpu().numpy()
    y_eval_cov = y_eval_tensor.covariance_matrix.cpu().detach().numpy()
    y_eval_var = y_eval_tensor.variance.detach().cpu().numpy()


    x_train, y_train, x_eval, y_eval, y_eval_err = \
            GPDataExporter(x_train, y_train, x_eval, y_eval_mean, y_eval_var, ScalerSet,
                        NORM_X_FLAG=NORM_X_FLAG, NORM_Y_FLAG=NORM_Y_FLAG)
        
    y_eval_full[eval_mask_list[i],:,:] = y_eval
    y_eval_err_full[eval_mask_list[i],:,:] = y_eval_err






2025-05-08 16:55:31.875 | INFO     | __main__:GPDataLoader:61 - 
x: (3,) 
y: (3, 202) 
x_pred(14,)
2025-05-08 16:55:33.026 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 100/200	Loss: -1.2300845384597778
2025-05-08 16:55:34.034 | INFO     | __main__:EISGPTrain:66 - C0 - Iter 200/200	Loss: -3.013937473297119
2025-05-08 16:55:34.054 | INFO     | __main__:GPDataExporter:99 - 
x: (3,) 
y: (3, 101, 2) 
x_pred(14,) 
y_pred(14, 101, 2) 
y_pred(14, 101, 2)
2025-05-08 16:55:34.055 | INFO     | __main__:GPDataLoader:61 - 
x: (3,) 
y: (3, 202) 
x_pred(14,)
2025-05-08 16:55:35.091 | INFO     | __main__:EISGPTrain:66 - C1 - Iter 100/200	Loss: -1.2432068586349487
2025-05-08 16:55:36.115 | INFO     | __main__:EISGPTrain:66 - C1 - Iter 200/200	Loss: -3.0036368370056152
2025-05-08 16:55:36.164 | INFO     | __main__:GPDataExporter:99 - 
x: (3,) 
y: (3, 101, 2) 
x_pred(14,) 
y_pred(14, 101, 2) 
y_pred(14, 101, 2)
2025-05-08 16:55:36.165 | INFO     | __main__:GPDataLoader:61 - 
x: (4,) 
y: (4, 202) 
x_pr

### Loss Plot

In [106]:
if True:
    fig, axis = plt.subplots(1+2*n_clusters,1,figsize=(6,6))


    for j in range(n_clusters):
        loss_inst = loss_list[j]
        noise_inst = noise_list[j]
        length_inst = length_list[j]

        axis[0].plot(loss_inst+10, label = f"Cluster {j}")
        axis[0].set_yscale('log')
        axis[0].set_title("Loss")


        cmap = plt.colormaps.get_cmap('viridis')
        for i in range(np.shape(length_inst)[1]):
        # for i in range(60,70):
        # for i in range(0,20):
            axis[1+2*j].plot(noise_inst[:,i].flatten(), color = cmap(i/np.shape(length_inst)[1]), linewidth=2, label=f"LengthScale {i+1}")
            axis[1+2*j+1].plot(length_inst[:,i].flatten(), color = cmap(i/np.shape(length_inst)[1]), linewidth=2, label=f"LengthScale {i+1}")
        
        # axis[1+2*j].set_yscale('log')
        axis[1+2*j].set_title(f"Noise C{j}")
        # axis[1+2*j+1].set_yscale('log')
        axis[1+2*j+1].set_title(f"LengthScale C{j}")

    # axis[0].legend(frameon=False, loc='upper right')
    fig.set_tight_layout(True)

### GPR Result Plot

In [107]:
if True:
    
    cluster_id = None
    if cluster_id is None:
        x_train = x_train_full
        y_train = y_train_full
        x_eval = x_eval_full
        y_eval = y_eval_full
        y_eval_err = y_eval_err_full

    else:
        x_train = x_train_full[train_mask_list[cluster_id]]
        y_train = y_train_full[train_mask_list[cluster_id],:,:]
        x_eval = x_eval_full[eval_mask_list[cluster_id]]
        y_eval = y_eval_full[eval_mask_list[cluster_id],:,:]
        y_eval_err = y_eval_err_full[eval_mask_list[cluster_id],:,:]


        # Scaler_Y_real   = StandardScaler()
        # Scaler_Y_imag   = StandardScaler()

        # if NORM_Y_FLAG:
        #     y_train[:,:,0] = Scaler_Y_real.fit_transform(y_train[:,:,0])
        #     y_train[:,:,1] = Scaler_Y_imag.fit_transform(y_train[:,:,1])
        #     y_eval[:,:,0] = Scaler_Y_real.transform(y_eval[:,:,0])
        #     y_eval[:,:,1] = Scaler_Y_imag.transform(y_eval[:,:,1])
            


    n_freq = np.shape(y_eval)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('viridis_r')
    axis0 = fig.add_subplot(2,2,1)
    axis1 = fig.add_subplot(2,2,2)
    axis2 = fig.add_subplot(2,2,3)
    axis3 = fig.add_subplot(2,2,4)


    # for i in range(n_freq):
    # for i in range(30,40):
    for i in range(np.shape(y_eval)[1]):
        axis0.fill_between(x_eval, y_eval[:,i,0] - 2*np.sqrt(y_eval_err[:,i,0]), y_eval[:,i,0] + 2*np.sqrt(y_eval_err[:,i,0]), 
                        alpha=0.2, color = cmap(i/n_freq))
        
        axis1.plot(x_eval, y_eval[:,i,0], color = cmap(i/n_freq))
        axis1.plot(x_train, y_train[:,i,0], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')
        axis0.plot(x_train, y_train[:,i,0], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')

        axis2.fill_between(x_eval, -(y_eval[:,i,1] - 2*np.sqrt(y_eval_err[:,i,1])), -(y_eval[:,i,1] + 2*np.sqrt(y_eval_err[:,i,1])), 
                        alpha=0.2, color = cmap(i/n_freq))
        
        axis3.plot(x_eval, -y_eval[:,i,1], color = cmap(i/n_freq))
        axis3.plot(x_train, -y_train[:,i,1], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')
        axis2.plot(x_train, -y_train[:,i,1], color = cmap(i/n_freq), linestyle = ' ', marker = 'o')
    if not LOG_FLAG:
        axis0.set_yscale('log')
        axis1.set_yscale('log')
        axis2.set_yscale('log')
        axis3.set_yscale('log')

    axis1.sharey(axis0)
    axis3.sharey(axis2)
    axis0.set_xlabel('x')
    axis0.set_ylabel('y')


### GPR EIS Plot

In [108]:
if True:   
    cluster_id = None
    if cluster_id is None:
        x_train = x_train_full
        y_train = y_train_full
        x_eval = x_eval_full
        y_eval = y_eval_full
        y_eval_err = y_eval_err_full

    else:
        x_train = x_train_full[train_mask_list[cluster_id]]
        y_train = y_train_full[train_mask_list[cluster_id],:,:]
        x_eval = x_eval_full[eval_mask_list[cluster_id]]
        y_eval = y_eval_full[eval_mask_list[cluster_id],:,:]
        y_eval_err = y_eval_err_full[eval_mask_list[cluster_id],:,:]

        # Scaler_Y_real   = StandardScaler()
        # Scaler_Y_imag   = StandardScaler()

        # if NORM_Y_FLAG:
        #     y_train[:,:,0] = Scaler_Y_real.fit_transform(y_train[:,:,0])
        #     y_train[:,:,1] = Scaler_Y_imag.fit_transform(y_train[:,:,1])
        #     y_eval[:,:,0] = Scaler_Y_real.transform(y_eval[:,:,0])
        #     y_eval[:,:,1] = Scaler_Y_imag.transform(y_eval[:,:,1])
            


    n_freq = np.shape(y_eval)[0]
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    axis0 = fig.add_subplot(2,3,1)
    axis1 = fig.add_subplot(2,3,2)
    axis2 = fig.add_subplot(2,3,3, projection = '3d')
    axis3 = fig.add_subplot(2,3,4)
    axis4 = fig.add_subplot(2,3,5)
    axis5 = fig.add_subplot(2,3,6, projection = '3d')


    init_elev = 40  # 仰角
    init_azim = 45  # 方位角
    init_roll = 0
    axis2.view_init(elev=init_elev, azim=init_azim, roll=init_roll)
    axis5.view_init(elev=init_elev, azim=init_azim, roll=init_roll)


    new_f = chData[0,0,:].take(run_list, axis=0)

    if LOG_FLAG:
        
        y_EIS_train =   np.exp(y_train[:,:,0] + 1j * y_train[:,:,1])
        y_EIS_eval = np.exp(y_eval[:,:,0] + 1j * y_eval[:,:,1])

        # f_plot = range(40,80)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=1)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=1)
        new_f = new_f.take(f_plot, axis=0)

        
        for i in range(np.shape(x_eval)[0]):
            axis0.fill_between(new_f, y_eval[i,:,0] - 2*np.sqrt(y_eval_err[i,:,0]), y_eval[i,:,0] + 2*np.sqrt(y_eval_err[i,:,0]), 
                    alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
            axis3.fill_between(new_f, y_eval[i,:,1] - 2*np.sqrt(y_eval_err[i,:,1]), y_eval[i,:,1] + 2*np.sqrt(y_eval_err[i,:,1]), 
                    alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
        axis0.set_xscale('log')
        axis0.set_yscale('log')
        axis3.set_xscale('log')
        # axis3.set_yscale('log')
        

    else:
        y_EIS_train = y_train[:,:,0] + 1j*y_train[:,:,1]
        y_EIS_eval = y_eval[:,:,0] + 1j*y_eval[:,:,1]

        # f_plot = range(40,60)
        f_plot = range(np.shape(new_f)[0])
        y_EIS_train = y_EIS_train.take(f_plot, axis=1)
        y_EIS_eval = y_EIS_eval.take(f_plot, axis=1)
        new_f = new_f.take(f_plot, axis=0)

        for i in range(np.shape(x_eval)[0]):
            axis0.loglog(new_f, (np.real(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
            axis3.loglog(new_f, (-np.imag(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
        
        


    for i in range(np.shape(x_eval)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_eval[i,:])), color = cmap(i/np.shape(x_eval)[0]))
        
        
    for i in range(np.shape(x_train)[0]):
        axis1.semilogx(new_f, np.log10(np.abs(y_EIS_train[i,:])), 'black', alpha = 0.3)
        axis4.semilogx(new_f, np.rad2deg(np.angle(y_EIS_train[i,:])), 'black', alpha = 0.3)
    


    _x = np.arange(np.shape(x_eval)[0])
    _y = np.log10(new_f).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis2.plot_surface(X, Y, np.log10(np.abs(y_EIS_eval[:,:])), cmap='viridis_r', alpha=0.8)
    axis5.plot_surface(X, Y, -np.rad2deg(np.angle(y_EIS_eval[:,:])), cmap='viridis_r', alpha=0.8)




# END